<a href="https://colab.research.google.com/github/Hirva-Bhagat/GooglePlaceReviewScraper/blob/master/Bigram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import nltk
nltk.download('reuters') # one time execution
nltk.download('punkt')  # one time execution

from nltk.corpus import reuters
import math
import random

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
dataset_sents = reuters.sents()
# print first sentence
dataset_sents[1]

# get list of words
dataset_words = reuters.words()
# print first word
dataset_words[:10]

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN']

In [10]:
data_sents = dataset_sents[:40000]
data_sents_test = dataset_sents[40000:]

In [11]:
# number of words in train data
num_words = 0
for sentence in data_sents:
  num_words += len(sentence)
num_words
# create two lists containing words
data_words_train = dataset_words[:num_words]
data_words_test = dataset_words[num_words:]

In [12]:
def createBigram(data):
	listOfBigrams = []
	bigramCounts = {}
	unigramCounts = {}

	for i in range(len(data)):
		if i < len(data) - 1:

			listOfBigrams.append((data[i], data[i + 1]))

			if (data[i], data[i+1]) in bigramCounts:
				bigramCounts[(data[i], data[i + 1])] += 1
			else:
				bigramCounts[(data[i], data[i + 1])] = 1

		if data[i] in unigramCounts:
			unigramCounts[data[i]] += 1
		else:
			unigramCounts[data[i]] = 1

	return listOfBigrams, unigramCounts, bigramCounts

In [13]:
def calcBigramProb(listOfBigrams, unigramCounts, bigramCounts):

	listOfProb = {}
	for bigram in listOfBigrams:
		word1 = bigram[0]
		word2 = bigram[1]
		
		listOfProb[bigram] = (bigramCounts.get(bigram))/(unigramCounts.get(word1))

	file = open('bigramProb.txt', 'w')
	file.write('Bigram' + '\t\t\t' + 'Count' + '\t' + 'Probability' + '\n')

	for bigrams in listOfBigrams:
		file.write(str(bigrams) + ' : ' + str(bigramCounts[bigrams]) + ' : ' + str(listOfProb[bigrams]) + '\n')
	file.close()

	return listOfProb

In [14]:
def bigramWithAddOneSmoothing(listOfBigrams, unigramCounts, bigramCounts):

	listOfProb = {}
	cStar = {}


	for bigram in listOfBigrams:
		word1 = bigram[0]
		word2 = bigram[1]
		listOfProb[bigram] = (bigramCounts.get(bigram) + 1)/(unigramCounts.get(word1) + len(unigramCounts))
		cStar[bigram] = (bigramCounts[bigram] + 1) * unigramCounts[word1] / (unigramCounts[word1] + len(unigramCounts))

	file = open('addOneSmoothing.txt', 'w')
	file.write('Bigram' + '\t\t\t' + 'Count' + '\t' + 'Probability' + '\n')

	for bigrams in listOfBigrams:
		file.write(str(bigrams) + ' : ' + str(bigramCounts[bigrams])
				   + ' : ' + str(listOfProb[bigrams]) + '\n')

	file.close()

	return listOfProb, cStar

In [15]:
# Main Program

# Create a list of bigrams and get frequencies of unigrams and bigrams
listOfBigrams, unigramCounts, bigramCounts = createBigram(data_words_train)

# Calculate bigram probabilities
bigramProb = calcBigramProb(listOfBigrams, unigramCounts, bigramCounts)

# Apply Add-1 Smoothing and calculate probabilities and get reconstructed count of bigrams
bigramAddOne, addOneCstar = bigramWithAddOneSmoothing(listOfBigrams, unigramCounts, bigramCounts)

In [16]:
input = 'we must be very careful'

inputList = [] # list to store bigrams

for i in range(len(input.split())-1):
  inputList.append((input.split()[i], input.split()[i+1]))
inputList

[('we', 'must'), ('must', 'be'), ('be', 'very'), ('very', 'careful')]

In [17]:
# Open a file to write output
output1 = open('bigramProb-OUTPUT.txt', 'w')

# initial probability of a sentence
outputProb1 = 1

output1.write('Bigram\t\t\t\t' + 'Count\t\t\t\t' + 'Probability\n\n')

for i in range(len(inputList)):

  # if bigram is present in the model, get updated probability
  if inputList[i] in bigramProb:
    # write bigram, its count and probability to the file
    output1.write(str(inputList[i]) + '\t\t' + str(bigramCounts[inputList[i]]) + '\t\t' + str(bigramProb[inputList[i]]) + '\n')
    # multiply with probability of a current bigram
    outputProb1 *= bigramProb[inputList[i]]

  # if bigram is not present in the model, sentence probability is zero
  else:
    output1.write(str(inputList[i]) + '\t\t\t' + str(0) + '\t\t\t' + str(0) + '\n')
    outputProb1 *= 0

output1.write('\n' + 'Probablility = ' + str(outputProb1))
outputProb1

2.8076159793430567e-07

In [18]:
# Open a file to write output
output2 = open('addOneSmoothing-OUTPUT.txt', 'w')

# initial probability of a sentence
outputProb2 = 1

output2.write('Bigram\t\t\t\t' + 'Count\t\t\t\t' + 'Probability\n\n')

for i in range(len(inputList)):

  # if bigram is present in the model, get updated probability
  if inputList[i] in bigramAddOne:
    # Update probability of the sentence
    outputProb2 *= bigramAddOne[inputList[i]]

    output2.write(str(inputList[i]) + '\t\t' + str(addOneCstar[inputList[i]]) + '\t\t' + str(bigramAddOne[inputList[i]]) + '\n')

  # if bigram is not present in the model, use unigram counts to get estimated probability
  else:
    # if first word in a bigram is not present in unigrams, add with with count 1
    if inputList[i][0] not in unigramCounts:
      unigramCounts[inputList[i][0]] = 1
    
    # calculate probability of that word
    prob = (1) / (unigramCounts[inputList[i][0]] + len(unigramCounts))

    # # reconstructed count for the bigram
    addOneCStar = 1 * unigramCounts[inputList[i][0]] / (unigramCounts[inputList[i][0]] + len(unigramCounts))
    
    # Update probability of the sentence
    outputProb2 *= prob

    output2.write(str(inputList[i]) + '\t' + str(addOneCStar) + '\t' + str(prob) + '\n')

output2.write('\n' + 'Probablility = ' + str(outputProb2))
outputProb2

4.2254340301928457e-14

In [21]:
# input sentence
print(input)

# list of bigrams
print(inputList)

# probability given by simple bigram model
print ('Bigram Model: ', outputProb1)

# probability given by bigram model with add-1 smoothing
print ('Add One: ', outputProb2)

we must be very careful
[('we', 'must'), ('must', 'be'), ('be', 'very'), ('very', 'careful')]
Bigram Model:  2.8076159793430567e-07
Add One:  4.2254340301928457e-14


In [22]:
def sentence_prob_with_next_word(next_word):
  outputProb = 1
  new_bigram = (input.split()[-1], next_word)
  if new_bigram in bigramAddOne:
    outputProb *= bigramAddOne[new_bigram]
  else:
    if new_bigram[0] not in unigramCounts:
      unigramCounts[new_bigram[0]] = 1
    prob = (1) / (unigramCounts[new_bigram[0]] + len(unigramCounts))
    outputProb *= prob
  return outputProb
input = 'the investors are'
possible_words = ['cheated', 'happy', 'smart', 'afraid']

inputList = []
outputProb = 1

for i in range(len(input.split())-1):
  inputList.append((input.split()[i], input.split()[i+1]))


for i in range(len(inputList)):

  if inputList[i] in bigramAddOne:
    outputProb *= bigramAddOne[inputList[i]]
  else:
    if inputList[i][0] not in unigramCounts:
      unigramCounts[inputList[i][0]] = 1
    prob = (1) / (unigramCounts[inputList[i][0]] + len(unigramCounts))
    outputProb *= prob

max_prob = 0
index_of_next_word = -1
for i, word in enumerate(possible_words):
  final_prob = outputProb * sentence_prob_with_next_word(word)
  if final_prob > max_prob:
    max_prob = final_prob
    index_of_next_word = i

print('Next Word:', possible_words[index_of_next_word])
print('Output Sentece:', input, possible_words[index_of_next_word])

Next Word: happy
Output Sentece: the investors are happy


In [23]:
input1 = 'the market is very happy these days'
input2 = 'market is the happy these very days'


inputList1 = []
inputList2 = []


outputProb1 = 1
outputProb2 = 1


for i in range(len(input1.split())-1):
  inputList1.append((input1.split()[i], input1.split()[i+1]))

for i in range(len(input2.split())-1):
  inputList2.append((input2.split()[i], input2.split()[i+1]))


for i in range(len(inputList1)):
  if inputList1[i] in bigramAddOne:
    outputProb1 *= bigramAddOne[inputList1[i]]
  else:
    if inputList1[i][0] not in unigramCounts:
      unigramCounts[inputList1[i][0]] = 1
    prob1 = (1) / (unigramCounts[inputList1[i][0]] + len(unigramCounts))
    outputProb1 *= prob1


for i in range(len(inputList2)):
  if inputList2[i] in bigramAddOne:
    outputProb2 *= bigramAddOne[inputList2[i]]
  else:
    if inputList2[i][0] not in unigramCounts:
      unigramCounts[inputList2[i][0]] = 1
    prob2 = (1) / (unigramCounts[inputList2[i][0]] + len(unigramCounts))
    outputProb2 *= prob2

print (input1, ':', outputProb1)
print (input2, ':', outputProb2)

the market is very happy these days : 3.0787233025784113e-22
market is the happy these very days : 2.5840603259051406e-24


In [24]:
# initial word
text = ["there"]

sentence_finished = False
 
while not sentence_finished:
  # select a random probability threshold  
  r = random.random()
  accumulator = 0.0

  for pair in bigramProb.keys():
    if pair[0] == text[-1]:
      accumulator += bigramProb[pair]
      # select words that are above the probability threshold
      if accumulator >= r:
          text.append(pair[1])
          break

  if text[-1] == 'None':
    sentence_finished = True
  if len(text) > 20:
    sentence_finished = True
 
print (' '.join([t for t in text if t]))

there was an affiliate Rank Xerox Corp said . 2 pct next six weeks in cash for the country . Exports


NameError: ignored